In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
from pathlib import Path
from loguru import logger
import sys
from new_modeling_toolkit.common.asset.plant import ResourceCategory
from new_modeling_toolkit.recap.recap_case import RecapCase
import new_modeling_toolkit.recap.updated_dispatch_model as dispatch_model
from new_modeling_toolkit.recap.updated_dispatch_model import _divide_monte_carlo_draws_into_subproblems, _compress_dispatch_subproblem_data, _construct_and_solve_dispatch_subproblem, UpdatedDispatchModel
from new_modeling_toolkit.utils.parallelization_utils import parallelize
from new_modeling_toolkit.utils.pyomo_utils import convert_pyomo_object_to_dataframe
from new_modeling_toolkit.common.util import DirStructure

import pyomo.environ as pyo

import time

In [ ]:
import importlib

importlib.reload(dispatch_model)

In [ ]:
dir_str = DirStructure(code_dir=Path("../new_modeling_toolkit/"), data_folder="data", model_name="recap")

In [ ]:
recap_case = RecapCase.from_dir(dir_str=dir_str, case_name="WALC_speedup_challenge_EUE", gurobi_credentials=None)

In [ ]:
perfect_capacity = 0

# Divide Monte Carlo draws into N-year long subproblems
data_by_draw_and_subproblem = _divide_monte_carlo_draws_into_subproblems(recap_case.monte_carlo_draws)

# Get inputs to compression
compress_subproblem_data_kwargs = [
    dict(
        draw_name=draw_name,
        subproblem_number=subproblem_number,
        subproblem_data=subproblem_data,
        perfect_capacity=perfect_capacity,
    )
    for draw_name, subproblem_number, subproblem_data in data_by_draw_and_subproblem
]

# Compress subproblem data
dispatch_model_data_list = parallelize(
    _compress_dispatch_subproblem_data,
    kwargs_list=compress_subproblem_data_kwargs,
    progress_bar_description="Subproblem Compression",
    num_processes=min(len(compress_subproblem_data_kwargs), 20),
)

# Construct and solve compressed dispatch subproblems
dispatch_model_kwargs = [
    dict(
        draw_name=draw_name,
        subproblem_number=subproblem_number,
        dispatch_model_data=dispatch_model_data,
        dispatch_objective=recap_case.case_settings.dispatch_objective,
    )
    for draw_name, subproblem_number, dispatch_model_data in dispatch_model_data_list
]

In [ ]:
args = dispatch_model_kwargs[0]

In [ ]:
dispatch_model_data = args["dispatch_model_data"]
dispatch_objective = args["dispatch_objective"]

construction_start = time.time()
dispatch_model_ = UpdatedDispatchModel(
    dispatch_model_data=dispatch_model_data,
    objective_fn=dispatch_objective,
)
construction_end = time.time()

print(construction_end - construction_start)

In [ ]:
solve_start = time.time()
solution = dispatch_model_.solve(solver_name="gurobi")
solve_end = time.time()

print(solve_end - solve_start)

In [ ]:
dispatch_model_._create_dispatch_dataframe()

In [ ]:
def _create_dispatch_dataframe(self) -> pd.DataFrame:
    """Creates a data frame containing dispatch information for each timestamp.

    Returns:
        dispatch_df: the data frame with dispatch information
    """
    # Get dispatch results for all resources
    Increase_Load_MW = convert_pyomo_object_to_dataframe(self.Increase_Load_MW)
    Provide_Power_MW = convert_pyomo_object_to_dataframe(self.Provide_Power_MW)
    Provide_Reserve_MW = convert_pyomo_object_to_dataframe(self.Provide_Reserve_MW)

    dispatch_results_by_resource = pd.concat([Increase_Load_MW, Provide_Power_MW, Provide_Reserve_MW], axis=1)

    dispatch_results_by_group = pd.concat(
        [
            dispatch_results_by_resource.loc[pd.IndexSlice[list(group), :], :]
            .groupby(self.DISPATCH_HOURS.name)
            .sum()
            .rename(columns=lambda col: f"{group_name}_{col}")
            for group_name, group in [
                ("Storage", self.STORAGE_RESOURCES),
                ("Hydro", self.HYDRO_RESOURCES),
                ("DR", self.DR_RESOURCES),
            ]
        ],
        axis=1,
    )

    # Create dispatch dataframe
    dispatch_df = pd.concat(
        {
            "unserved_energy": convert_pyomo_object_to_dataframe(self.Unserved_Energy).squeeze(),
            "unserved_reserve": convert_pyomo_object_to_dataframe(self.Unserved_Reserve).squeeze(),
            "net_load": convert_pyomo_object_to_dataframe(self.Net_Load).squeeze(),
            "operating_reserve": convert_pyomo_object_to_dataframe(self.Operating_Reserve).squeeze(),
        },
        axis=1,
    )
    dispatch_df = pd.concat([dispatch_df, dispatch_results_by_group], axis=1)
    dispatch_df.loc[:, "perfect_capacity"] = pyo.value(self.Perfect_Capacity_MW)
    dispatch_df.loc[:, "unserved_energy_and_reserve"] = dispatch_df.loc[
        :, ["unserved_energy", "unserved_reserve"]
    ].sum(axis=1)

    # Upsample to original (full) length of subproblem
    dispatch_df_full = pd.DataFrame(index=self.all_hours)
    dispatch_df_full = dispatch_df_full.join(dispatch_df)
    dispatch_df_full[["unserved_energy", "unserved_reserve", "unserved_energy_and_reserve"]] =\
        dispatch_df_full[["unserved_energy", "unserved_reserve", "unserved_energy_and_reserve"]].fillna(0)

    # Set index
    dispatch_df_full = dispatch_df_full.reset_index(drop=True).set_index(self.all_timestamps)
    dispatch_df_full.index.name = dispatch_model._NET_LOAD_TIMESTAMP_INDEX_NAME
    
    return dispatch_df_full

In [ ]:
dispatch_results = _create_dispatch_dataframe(dispatch_model_)
dispatch_results

In [ ]:
dispatch_results.loc[(dispatch_results["Storage_Provide_Power_MW"] > 0) | (dispatch_results["Storage_Increase_Load_MW"] > 0)]

In [ ]:
dispatch_results[["unserved_energy", "unserved_reserve"]].sum()

In [ ]:
dispatch_results["net_load"].max()

In [ ]:
args = dispatch_model_kwargs[0]
subproblem_results = _construct_and_solve_dispatch_subproblem(**args)

In [ ]:
dispatch_model_data = args["dispatch_model_data"]

In [ ]:
dispatch_model_.all_hours = dispatch_model_data.all_hours

In [ ]:
from pydantic import BaseModel
from pydantic import Field
from typing import Optional

In [ ]:
class Test(BaseModel):
    
    field1: int
    field2: int
    field3: int = Field(freq="M")
    
    d1: dict
    d2: dict
    #_d: Optional[dict]
        
    @property
    def d(self):
        return {**self.d1, **self.d2}

In [ ]:
t = Test(
    field1=1,
    field2=2,
    field3=3,
    d1={"a": 1}, 
    d2={"b": 2}
)

In [ ]:
{**t.d1}

In [ ]:
d1 = {"a": {1: 10, 2: 12, 3: 9}, "b": {1: 14, 2: 16, 3: 19}}

d2 = {"c": {1: 10, 2: 12, 3: 9}, "d": {1: 10, 2: 12, 3: 9}}

d = {**d1, **d2}

d

In [ ]:
d1 = {i: f for i, f in zip(np.arange(100000), np.random.rand(100000))}
d2 = {i: f for i, f in zip(np.arange(100000, 200000), np.random.rand(100000))}

In [ ]:
logger.remove()
logger.add(sys.stdout, level="WARNING")

# Algorithm

* Find all continuous periods with net load >= 0

* Apply a buffer period to start and end of each window (1 week for now)

* Group all windows together that are separated by less than desired (2 weeks for now)

* Determine which window should be the "first" window, by finding the earliest window with sufficient lead time

In [ ]:
_START_DATETIME_COLNAME = "start_datetime"
_END_DATETIME_COLNAME = "end_datetime"
_PERIOD_ID = "period_id"
_DISPATCH_PROBLEM_ID = "dispatch_problem_id"

In [ ]:
_PERIOD_START_BUFFER = pd.Timedelta(hours=-168)
_PERIOD_END_BUFFER = pd.Timedelta(hours=168)
_PERIOD_MIN_SEPARATION_WINDOW = pd.Timedelta(hours=168*2)
_PERIOD_MIN_LEAD_WINDOW = pd.Timedelta(hours=168*2)

# Create Input Data

## Old: Fake Data

### Raw Negative Time Series

In [ ]:
# dt_index_raw = pd.date_range(start="2010-01-01 00:00", end="2020-01-01 00:00", inclusive="left", freq="H")

In [ ]:
# neg_series_raw = pd.Series(index=dt_index_raw, data=-1.0, name="net_load")

### Remove Leap Days

To verify that the code works without leap days

In [ ]:
# input_raw = pd.concat({"MC_draw_1": neg_series_raw, "MC_draw_2": neg_series_raw}, axis=0, names=("MC_draw", "timestamp"))

### Remove Leap Days

To verify that the code works without leap days

In [ ]:
# input_raw = input_raw.loc[~((input_raw.index.get_level_values("timestamp").month == 2) & (input_raw.index.get_level_values("timestamp").day == 29))]

### Define Positive-Net-Load Periods

In [ ]:
# input_data = input_raw.copy()

In [ ]:
# input_data.loc[pd.IndexSlice["MC_draw_1", "2013-03-10 06:00":"2013-03-12 19:00"]] = 5.0
# input_data.loc[pd.IndexSlice["MC_draw_1", "2014-07-24 04:00":"2014-07-29 22:00"]] = 5.0
# input_data.loc[pd.IndexSlice["MC_draw_1", "2014-08-12 07:00":"2014-08-15 15:00"]] = 5.0
# input_data.loc[pd.IndexSlice["MC_draw_1", "2018-05-04 00:00"]] = 5.0
# input_data.loc[pd.IndexSlice["MC_draw_1", "2018-06-12 10:00":"2018-06-12 12:00"]] = 5.0
# input_data.loc[pd.IndexSlice["MC_draw_1", "2018-06-18 10:00":"2018-06-18 12:00"]] = 5.0
# input_data.loc[pd.IndexSlice["MC_draw_2", "2017-01-01 09:00":"2017-01-02 14:00"]] = 5.0
# input_data.loc[pd.IndexSlice["MC_draw_2", "2017-01-18 02:00":"2017-01-18 10:00"]] = 5.0
# input_data.loc[pd.IndexSlice["MC_draw_2", "2017-12-24 13:00":"2017-12-26 12:00"]] = 5.0
# input_data.loc[pd.IndexSlice["MC_draw_2", "2018-04-24 13:00":"2018-04-25 12:00"]] = 5.0

In [ ]:
# input_data.index = pd.MultiIndex.from_arrays([input_data.index.get_level_values(0), input_data.index.get_level_values(1).year, input_data.index.get_level_values(1)], names=("MC_draw", "year", "timestamp"))

In [ ]:
# input_data

## New: Use MC Draws

In [ ]:
dir_str = DirStructure(code_dir=Path("../new_modeling_toolkit/"), data_folder="data", model_name="recap")

In [ ]:
recap_case = RecapCase.from_dir(dir_str=dir_str, case_name="WALC_speedup_benchmark_EUE", gurobi_credentials=None)

In [ ]:
input_data = pd.concat({draw_id: draw.net_load for draw_id, draw in recap_case.monte_carlo_draws.items()}, axis=0, names=("MC_draw", "timestamp")).rename("net_load")

In [ ]:
input_data.index = pd.MultiIndex.from_arrays([input_data.index.get_level_values(0), (input_data.index.get_level_values(1).year - input_data.index.get_level_values(1).year.min()) // 12, input_data.index.get_level_values(1)], names=("MC_draw", "subproblem_ID", "timestamp"))

In [ ]:
input_data

In [ ]:
# Define search interval upper/lower bounds
UB = max(recap_case.monte_carlo_draws["MC_draw_0"].load + recap_case.monte_carlo_draws["MC_draw_0"].reserves)
LB = -UB

logger.debug(f"Initial Lower Bound: {LB:.2f} MW")
logger.debug(f"Initial Upper Bound: {UB:.2f} MW")

# Define reliability as function of perfect capacity when ELRs are effectively removed from system
def reliability_func_no_ELRs(perfect_capacity):
    unserved_energy_and_reserve = recap_case.run_dispatch_no_ELRs(perfect_capacity=perfect_capacity)
    reliability = recap_case.calculate_reliability(unserved_energy_and_reserve, metric="EUE")
    return reliability

# Use bisection method to get perfect capacity shortfall lower and upper bounds
perfect_capacity_UB = recap_case.bisection_method(
    reliability_func=reliability_func_no_ELRs, target=0.1, LB=LB, UB=UB
)
perfect_capacity_LB = perfect_capacity_UB - recap_case.ELR_capacity  # ELR ELCC upper bound is 100%

In [ ]:
perfect_capacity_UB

In [ ]:
perfect_capacity_LB

In [ ]:
# input_data = input_data - perfect_capacity_LB
#input_data = input_data + 600

In [ ]:
input_data

# Algorithm Implementation

In [ ]:
_PERIOD_START_BUFFER = pd.Timedelta(hours=-168)
_PERIOD_END_BUFFER = pd.Timedelta(hours=168)
_PERIOD_MIN_SEPARATION_WINDOW = pd.Timedelta(hours=168*2)
_PERIOD_CONSECUTIVE_DELTA = pd.Timedelta(hours=24)

In [ ]:
subproblems = _divide_monte_carlo_draws_into_subproblems(recap_case.monte_carlo_draws)

In [ ]:
subproblem = subproblems[0]
subproblem_data_dict = subproblem[2]
net_load = subproblem_data_dict["net_load"]

perfect_capacity = 0

In [ ]:
# Charlie's second attempt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# We just need a list of dates that are "in" and "out", and then label them by adjacency in the final step

net_load = net_load - perfect_capacity

# Get single subproblem
timesteps = net_load.index
dates = np.unique(timesteps.date)
span = dates.max() - dates.min()

# Get positive net load dates
pos_timesteps = timesteps[net_load.values > 0]
pos_dates = np.unique(pos_timesteps.date)

# Loop through pos_dates to get in dates
in_dates = pd.DatetimeIndex([]) # Initialize

# Add +/- 1 week buffer
for date in pos_dates:
    
    if date <= in_dates.max().date():
        #print("skipping ", date)
        continue

    start, end = max(date + _PERIOD_START_BUFFER, dates.min()), min(date + _PERIOD_END_BUFFER, dates.max())
    window_dates = pd.date_range(start, end, freq="D")
    in_dates = in_dates.union(window_dates)
    
# For testing edge case of dispatch window crossing boundary of loop
#in_dates = in_dates.insert(0, pd.Timestamp(year=1979, month=1, day=10))
#in_dates = in_dates.insert(-1, pd.Timestamp(year=1990, month=12, day=30))

# Consolidate windows < 2 weeks apart
in_dates = in_dates.sort_values()
i = 0   
while i < len(in_dates) - 1:
    start, end = in_dates[i], in_dates[i+1]
    time_delta = end - start
    if (time_delta > _PERIOD_CONSECUTIVE_DELTA) and (time_delta < _PERIOD_MIN_SEPARATION_WINDOW): # non-consecutive dates < 2 weeks apart
        #print(f"Consolidating {start} through {end}")
        window_dates = pd.date_range(start, end, freq="D")
        in_dates = in_dates.union(window_dates)
    i += 1
        
# Use modular arithmetic to consolidate first and last periods if < 2 weeks apart in "loop"
start, end = in_dates[-1], in_dates[0]
time_delta = end - start
time_delta = pd.Timedelta(seconds=np.mod(time_delta.total_seconds(), span.total_seconds()))
if (time_delta < _PERIOD_MIN_SEPARATION_WINDOW):
    #print(f"Consolidating {start} through {end}")
    window_dates = pd.date_range(start, dates.max(), freq="D")
    in_dates = in_dates.union(window_dates)
    window_dates = pd.date_range(dates.min(), end, freq="D")
    in_dates = in_dates.union(window_dates)

# Label windows
df_in = pd.DataFrame(index=pd.DatetimeIndex(dates))
df_in.loc[in_dates, "in"] = 1
df_in = df_in.fillna(0)
df_in["label"] = (df_in.shift(1) > df_in).cumsum() + 1

# Join period after last window with first window
df_in.loc[df_in["label"] == df_in["label"].max(), "label"] = df_in["label"].min()

# Re-sample at hourly frequency
df_in.loc[df_in.index[-1] + pd.Timedelta(hours=24)] = df_in.loc[df_in.index[-1]]
df_in = df_in.resample("H").ffill()[:-1]

In [ ]:
timestamp_to_hour_mapping = {ts: i for i, ts in enumerate(df_in.index)}
timestamp_to_day_mapping = {ts: i for i, ts in enumerate(df_in.index.to_period("D").to_timestamp().unique())}
timestamp_to_month_mapping = {ts: i for i, ts in enumerate(df_in.index.to_period("M").to_timestamp().unique())}
timestamp_to_year_mapping = {ts: i for i, ts in enumerate(df_in.index.to_period("Y").to_timestamp().unique())}

df_in = df_in[["in"]]

df_in["hour"] = range(len(df_in))
df_in["day"] = list(timestamp_to_day_mapping[ts] for ts in df_in.index.to_period("D").to_timestamp())
df_in["month"] = list(timestamp_to_month_mapping[ts] for ts in df_in.index.to_period("M").to_timestamp())
df_in["year"] = list(timestamp_to_year_mapping[ts] for ts in df_in.index.to_period("Y").to_timestamp())

df_in_sub = df_in.loc[df_in["in"] == 1, ["hour", "day", "month", "year"]]

df_daily = df_in_sub.groupby("day")
days = list(df_daily.groups.keys())
day_to_hour_mapping = {d: list(df_daily.get_group(d)["hour"].values) for d in days}

df_monthly = df_in_sub.groupby("month")
months = list(df_monthly.groups.keys())
month_to_hour_mapping = {m: list(df_monthly.get_group(m)["hour"].values) for m in months}

df_annual = df_in_sub.groupby("year")
years = list(df_annual.groups.keys())
year_to_hour_mapping = {y: list(df_annual.get_group(y)["hour"].values) for y in years}

In [ ]:
timestamp_to_hour_mapping

In [ ]:
groups = df_in_hourly.groupby(df_in_hourly.index.to_period("D").to_timestamp()).groups
day_to_hour_mapping = {timestamp_to_day_mapping.loc[group]: list(timestamp_to_hour_mapping.loc[groups[group]].values) for group in groups}

groups = df_in_hourly.groupby(df_in_hourly.index.to_period("M").to_timestamp()).groups
month_to_hour_mapping = {timestamp_to_month_mapping.loc[group]: list(timestamp_to_hour_mapping.loc[groups[group]].values) for group in groups}

groups = df_in_hourly.groupby(df_in_hourly.index.to_period("Y").to_timestamp()).groups
year_to_hour_mapping = {timestamp_to_year_mapping.loc[group]: list(timestamp_to_hour_mapping.loc[groups[group]].values) for group in groups}

In [ ]:
df_in.loc[df_in["in"] == 1, "hour"]

In [ ]:
df_in["label"].plot()
(df_in["label"] * df_in["in"]).plot()

In [ ]:
# Adjust hydro budgets

hydro_pmin = subproblem_data_dict["hydro_pmin"]
hydro_pmin_out = hydro_pmin.multiply((1 - df_in["in"]).values, axis=0)

hydro_pmin_out_monthly = hydro_pmin_out.groupby([hydro_pmin_out.index.to_period("M").to_timestamp()]).sum()
hydro_budget_monthly = subproblem_data_dict["hydro_budget_monthly"]
hydro_budget_monthly_adjusted = hydro_budget_monthly - hydro_pmin_out_monthly
subproblem_data_dict["hydro_budget_monthly"] = hydro_budget_monthly_adjusted

hydro_pmin_out_annual = hydro_pmin_out.groupby([hydro_pmin_out.index.to_period("Y").to_timestamp()]).sum()
hydro_budget_annual = subproblem_data_dict["hydro_budget_annual"]
hydro_budget_annual_adjusted = hydro_budget_annual - hydro_pmin_out_annual
subproblem_data_dict["hydro_budget_annual"] = hydro_budget_annual_adjusted

In [ ]:
# Calculate storage initial SOC for each dispatch window

# Get available energy profile
available_energy = - net_load + perfect_capacity + hydro_pmin_out.sum(axis=1)

# Get storage pmax and SOC max
storage_pmax = subproblem_data_dict["storage_pmax"]
storage_SOC_max = subproblem_data_dict["storage_soc_max"]
storage_data_dict = subproblem_data_dict["storage_data_dict"]

# Sort storage resources by duration (ignore "inf")
durations = (storage_pmax.max(axis=0) / storage_SOC_max.max(axis=0)).replace(np.inf, np.nan).dropna().sort_values()

# Define dictionary of time series
timeseries = {
    "df_in": df_in,
    "available_energy": available_energy,
    "storage_pmax": storage_pmax,
    "storage_SOC_max": storage_SOC_max,
}

# Re-order time series by dispatch window label
first_window = df_in["label"].min()
last_window = df_in["label"].max()
loop_inds = (df_in["label"] == first_window) & (df_in.index > max(df_in.loc[df_in["label"] == last_window].index))
for ts in timeseries.keys():
    #print(ts)
    df_ts = timeseries[ts]
    df_ts = pd.concat([df_ts[loop_inds], df_ts[~loop_inds]])
    timeseries[ts] = df_ts
(df_in, available_energy, storage_pmax, storage_SOC_max) = timeseries.values()

# Group time series by dispatch window label
for ts in timeseries.keys():
    #print(ts)
    df_ts = timeseries[ts]
    df_ts = df_ts.groupby(df_in["label"])
    timeseries[ts] = df_ts
(df_in, available_energy, storage_pmax, storage_SOC_max) = timeseries.values()

# Loop through windows and storage resources; get initial SOC of each storage resource for each dispatch window
df_initial_storage_SOC = pd.DataFrame(columns=durations.index)
for window in available_energy.groups:
    
    # Get time series for dispatch window
    df_in_window = df_in.get_group(window)
    available_energy_window = available_energy.get_group(window)
    storage_pmax_window = storage_pmax.get_group(window)
    storage_SOC_max_window = storage_SOC_max.get_group(window)

    # Get first timestep of dispatch window
    window_start = df_in_window.index[df_in_window["in"].shift(1) < df_in_window["in"]]
    assert len(window_start) == 1
    window_start = window_start[0]
    
    # Charge resources in order of duration (shortest to longest)
    for resource in durations.index:

        # Calculate charging for resource
        charging = storage_pmax_window[resource].clip(upper=available_energy_window)
        SOC = charging.multiply(storage_data_dict[resource]["charging_efficiency"]).shift(1).fillna(0).cumsum().clip(upper=storage_SOC_max_window[resource])
        charging_final = charging.clip(upper=storage_SOC_max_window[resource] - SOC)

        # Update available energy
        available_energy_window -= charging_final
        
        # Save initial SOC of storage resource
        df_initial_storage_SOC.loc[window_start, resource] = SOC.loc[window_start]

In [ ]:
df_initial_storage_SOC["hour"] = timestamp_to_hour_mapping.loc[df_initial_storage_SOC.index]
df_initial_storage_SOC = df_initial_storage_SOC.reset_index(drop=True).set_index(["hour"])
df_initial_storage_SOC.to_dict()

In [ ]:
df_initial_storage_SOC.to_dict()

## Identify Positive Net Load Periods

In [ ]:
pd.set_option("display.max_rows", 150)

In [ ]:
def find_positive_net_load_periods(series: pd.Series):
    non_neg_series = series >= 0
    non_neg_groups = (non_neg_series != non_neg_series.shift(1)).cumsum() * non_neg_series
    starts_and_ends = non_neg_series.groupby(non_neg_groups).apply(lambda s: (s.index.get_level_values("timestamp").min(), s.index.get_level_values("timestamp").max()))
    starts_and_ends = starts_and_ends.drop(0, axis=0).reset_index(drop=True).rename_axis(index=_PERIOD_ID)
    starts_and_ends = pd.DataFrame(index=starts_and_ends.index, data=starts_and_ends.values.tolist(), columns=[_START_DATETIME_COLNAME, _END_DATETIME_COLNAME])
    
    return starts_and_ends

In [ ]:
start_and_end_times = input_data.groupby(["MC_draw", "year"]).apply(find_positive_net_load_periods)

In [ ]:
start_and_end_times

## Add Buffer Period

In [ ]:
start_and_end_times_with_buffer = start_and_end_times.copy()
start_and_end_times_with_buffer.loc[:, _START_DATETIME_COLNAME] = (start_and_end_times_with_buffer.loc[:, _START_DATETIME_COLNAME] + _PERIOD_START_BUFFER).dt.floor("D")
start_and_end_times_with_buffer.loc[:, _END_DATETIME_COLNAME] = (start_and_end_times_with_buffer.loc[:, _END_DATETIME_COLNAME] + _PERIOD_END_BUFFER).dt.ceil("D") - pd.Timedelta(hours=1)

In [ ]:
pd.concat({"No Buffer": start_and_end_times, "With Buffer": start_and_end_times_with_buffer}, axis=1)

## Combine Overlapping Periods

In [ ]:
def combine_overlapping_windows(frame: pd.DataFrame):
    start_date = frame.loc[:, _START_DATETIME_COLNAME].iloc[0]
    end_date = frame.loc[:, _END_DATETIME_COLNAME].iloc[0]
    period_id = 0
    window_id = 0
    
    results = pd.DataFrame(index=pd.Index([period_id], name=_PERIOD_ID), columns=[_START_DATETIME_COLNAME, _END_DATETIME_COLNAME])
    
    for index, row in frame.iloc[1:, :].iterrows():
        # print("curr start date: ", start_date)
        # print("curr end date: ", end_date)
        # print("period id: ", period_id )
        # display(row)
        if row[_START_DATETIME_COLNAME] < end_date:
            end_date = row[_END_DATETIME_COLNAME]
        else:
            # display(period_id)
            results.loc[period_id, _START_DATETIME_COLNAME] = start_date
            results.loc[period_id, _END_DATETIME_COLNAME] = end_date
            # results.loc[period_id, _DISPATCH_PROBLEM_ID] = window_id
            period_id += 1
            # if (row[_START_DATETIME_COLNAME] - _PERIOD_MIN_SEPARATION_WINDOW) > end_date:
            #     window_id += 1
                
            start_date = row[_START_DATETIME_COLNAME]
            end_date = row[_END_DATETIME_COLNAME]
            # display(results)
        # print()
    
    results.loc[period_id, _START_DATETIME_COLNAME] = start_date
    results.loc[period_id, _END_DATETIME_COLNAME] = end_date
    # results.loc[period_id, _DISPATCH_PROBLEM_ID] = window_id
    
    return results

In [ ]:
grouped_start_and_ends = start_and_end_times_with_buffer.groupby(["MC_draw", "year"]).apply(combine_overlapping_windows)

In [ ]:
grouped_start_and_ends

## Find Start Date

In [ ]:
def find_dispatch_problem_start_timestamp(frame):
    if frame[_END_DATETIME_COLNAME].min() > frame[_END_DATETIME_COLNAME].min().replace(month=1, day=1, hour=0) + _PERIOD_MIN_LEAD_WINDOW:
        start_timestamp = frame[_END_DATETIME_COLNAME].min().replace(month=1, day=1, hour=0)
    else:
        start_timestamp = frame[_END_DATETIME_COLNAME].min() + pd.Timedelta(hours=1)
    
    return start_timestamp

In [ ]:
start_timestamps = grouped_start_and_ends.groupby(["MC_draw", "year"]).apply(find_dispatch_problem_start_timestamp)

In [ ]:
start_timestamps

## Ordering Hours

In [ ]:
def create_timestamp_mapping(start_date):
    index = pd.date_range(start=start_date, periods=8760, freq="H")
    series = pd.Series(index=index, data=list(range(len(index))))
    series.index = series.index.map(lambda ts: ts.replace(year=start_date.year))
    series = series.sort_index()
    
    return series

In [ ]:
timestamp_numbers = start_timestamps.apply(create_timestamp_mapping).T.rename_axis(index="timestamp").stack(["MC_draw", "year"]).reorder_levels(["MC_draw", "year", "timestamp"]).rename("timestamp").astype(int)

In [ ]:
# pd.concat([input_data, timestamp_numbers], axis=1).sort_index()

# Comparing Stats

In [ ]:
grouped_start_and_ends.loc[:, "problem_length"] = (grouped_start_and_ends["end_datetime"] - grouped_start_and_ends["start_datetime"]) / pd.Timedelta(hours=1)
grouped_start_and_ends = grouped_start_and_ends.astype({"problem_length": int})

In [ ]:
stats_by_draw_and_year = grouped_start_and_ends.groupby(["MC_draw", "year"])["problem_length"].agg(["sum", "count"]).astype(int).rename({"sum": "Dispatch Hours per Year", "count": "Dispatch Events per Year"}, axis=1)

In [ ]:
stats_by_draw_and_year.groupby("MC_draw").describe().drop(["count", "std"], axis=1, level=1).round(1)

In [ ]:
plot_series = pd.Series(index=[pd.Timestamp("1979-01-01 00:00"), pd.Timestamp("2021-12-31 23:00")], data=0)
for idx, row in grouped_start_and_ends.xs("MC_draw_0").iterrows():
    plot_series.loc[row[_START_DATETIME_COLNAME]] = 1.0
    plot_series.loc[row[_END_DATETIME_COLNAME]] = 0.0
plot_series = plot_series.sort_index()

In [ ]:
plot_series_original  = pd.Series(index=[pd.Timestamp("1979-01-01 00:00"), pd.Timestamp("2021-12-31 23:00")], data=0)
for idx, row in start_and_end_times.xs("MC_draw_0").iterrows():
    plot_series_original.loc[row[_START_DATETIME_COLNAME]] = 1.0
    if row[_END_DATETIME_COLNAME] == row[_START_DATETIME_COLNAME]:
        plot_series_original.loc[row[_END_DATETIME_COLNAME] + pd.Timedelta(hours=1)] = 0.0
    else:
        plot_series_original.loc[row[_END_DATETIME_COLNAME]] = 0.0

In [ ]:
plot_series_original

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.FigureWidget()
fig.add_scatter(
    x=plot_series.index,
    y=plot_series,
    name="Dispatch Windows",
    line=dict(shape="hv"),
    fill="tozeroy"
)
fig.add_scatter(
    x=plot_series_original.index,
    y=plot_series_original,
    name="Positive Net Load Periods",
    line=dict(shape="hv"),
    fill="tozeroy"
)
fig.update_layout(height=750)

In [ ]:
fig.update_xaxes(range=["1984-01-01", "1986-01-01"])

In [ ]:
fig.update_xaxes(range=["1984-06-10", "1984-07-25"])